In [ ]:
# Task: JSON Parsing
# write a Python script that reads the students.jon JSON file and prints details of each student.

In [ ]:
[
    {"name": "Alice", "age": 20, "grade": "A"},
    {"name": "Bob", "age": 22, "grade": "B"},
    {"name": "Charlie", "age": 21, "grade": "A"}
]


In [ ]:
import json

def read_students(filename):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            students = json.load(f)

        print("Student Details:\n")
        for student in students:
            name = student.get("name", "N/A")
            age = student.get("age", "N/A")
            grade = student.get("grade", "N/A")
            print(f"Name: {name}, Age: {age}, Grade: {grade}")

    except FileNotFoundError:
        print(f"❌ File '{filename}' not found.")
    except json.JSONDecodeError:
        print("❌ Error: Invalid JSON format.")

if __name__ == "__main__":
    read_students("students.json")


In [ ]:
# Task: Weather API
# Use this url : https://openweathermap.org/
# Use the requests library to fetch weather data for a specific city(ex. your hometown: Tashkent) 
# and print relevant information (temperature, humidity, etc.).

In [ ]:
import requests

def get_weather(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        
        city_name = data["name"]
        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        weather_desc = data["weather"][0]["description"]
        
        print(f"Weather in {city_name}:")
        print(f"🌡️ Temperature: {temp}°C")
        print(f"💧 Humidity: {humidity}%")
        print(f"☁️ Condition: {weather_desc}")
    else:
        print("❌ Failed to fetch weather data. Check city name or API key.")

if __name__ == "__main__":
    city = "Tashkent"   # you can change this
    api_key = "YOUR_API_KEY"  # replace with your OpenWeather API key
    get_weather(city, api_key)


In [ ]:
# Task: JSON Modification
# Write a program that allows users to add new books, update existing book information, 
# and delete books from the books.json JSON file.

In [ ]:
[
    {"id": 1, "title": "1984", "author": "George Orwell"},
    {"id": 2, "title": "To Kill a Mockingbird", "author": "Harper Lee"}
]


In [ ]:
import json
import os

FILENAME = "books.json"

def load_books():
    if os.path.exists(FILENAME):
        with open(FILENAME, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_books(books):
    with open(FILENAME, "w", encoding="utf-8") as f:
        json.dump(books, f, indent=4)

def add_book(title, author):
    books = load_books()
    new_id = max([book["id"] for book in books], default=0) + 1
    books.append({"id": new_id, "title": title, "author": author})
    save_books(books)
    print(f"✅ Book '{title}' added.")

def update_book(book_id, title=None, author=None):
    books = load_books()
    for book in books:
        if book["id"] == book_id:
            if title:
                book["title"] = title
            if author:
                book["author"] = author
            save_books(books)
            print(f"✏️ Book with ID {book_id} updated.")
            return
    print("❌ Book not found.")

def delete_book(book_id):
    books = load_books()
    updated_books = [book for book in books if book["id"] != book_id]
    if len(updated_books) < len(books):
        save_books(updated_books)
        print(f"❌ Book with ID {book_id} deleted.")
    else:
        print("❌ Book not found.")

def list_books():
    books = load_books()
    if not books:
        print("📂 No books found.")
    else:
        print("\nCurrent Books:")
        for book in books:
            print(f"- ID: {book['id']}, Title: {book['title']}, Author: {book['author']}")
        print()

if __name__ == "__main__":
    while True:
        print("\n--- Book Manager ---")
        print("1. List Books")
        print("2. Add Book")
        print("3. Update Book")
        print("4. Delete Book")
        print("5. Exit")

        choice = input("Choose an option: ")

        if choice == "1":
            list_books()
        elif choice == "2":
            title = input("Enter book title: ")
            author = input("Enter author: ")
            add_book(title, author)
        elif choice == "3":
            book_id = int(input("Enter book ID to update: "))
            title = input("New title (leave blank to keep unchanged): ")
            author = input("New author (leave blank to keep unchanged): ")
            update_book(book_id, title if title else None, author if author else None)
        elif choice == "4":
            book_id = int(input("Enter book ID to delete: "))
            delete_book(book_id)
        elif choice == "5":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice. Try again.")


In [ ]:
# Task: Movie Recommendation System
# Use this url http://www.omdbapi.com/ to fetch information about movies.
# Create a program that asks users for a movie genre and recommends a random movie from that genre.

In [ ]:
import requests
import random

API_KEY = "YOUR_API_KEY"  # replace with your OMDb API key
BASE_URL = "http://www.omdbapi.com/"

def search_movies_by_genre(genre, max_results=50):
    """
    Since OMDb does not support direct genre search,
    we'll search with a general keyword and then filter by genre.
    """
    movies = []
    for page in range(1, 6):  # OMDb allows pagination up to 10 results per page
        params = {
            "apikey": API_KEY,
            "s": genre,   # search keyword
            "type": "movie",
            "page": page
        }
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        if "Search" in data:
            for item in data["Search"]:
                # Fetch full details to filter by actual genre
                movie_id = item["imdbID"]
                details = requests.get(BASE_URL, params={"apikey": API_KEY, "i": movie_id}).json()
                if "Genre" in details and genre.lower() in details["Genre"].lower():
                    movies.append(details)
        else:
            break
    return movies

def recommend_movie(genre):
    movies = search_movies_by_genre(genre)
    if not movies:
        print(f"❌ No movies found for genre: {genre}")
        return
    movie = random.choice(movies)
    print("\n🎬 Recommended Movie 🎬")
    print(f"Title: {movie['Title']}")
    print(f"Year: {movie['Year']}")
    print(f"Genre: {movie['Genre']}")
    print(f"IMDB Rating: {movie.get('imdbRating', 'N/A')}")
    print(f"Plot: {movie.get('Plot', 'N/A')}")


if __name__ == "__main__":
    genre = input("Enter a movie genre (e.g., Action, Comedy, Drama): ")
    recommend_movie(genre)


In [ ]:
# Enter a movie genre (e.g., Action, Comedy, Drama): Action

# 🎬 Recommended Movie 🎬
# Title: Mad Max: Fury Road
# Year: 2015
# Genre: Action, Adventure, Sci-Fi
# IMDB Rating: 8.1
# Plot: In a post-apocalyptic wasteland, Max teams up with a mysterious woman to flee from a warlord...
